In [97]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime

In [99]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-09-20 15:13:13.383214


In [79]:
save_data = True
tmp_data = False

In [80]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "Dstlnu_Hc_corr_BsigX_separation_dataRun1/"   
root_subdir = "axheim_data2_MC14_100kEvts/"   

root_path = nfs_path + "createBranchSeparatorData/" + root_subdir

In [81]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"

In [82]:
fileY4S = uproot.open(root_path + merged + "DXtagDstl.root:variables")

In [83]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename = root_path + merged + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/createBranchSeparatorData/axheim_data2_MC14_100kEvts/merged_muons.root:variables


In [84]:
df_FSPs = pd.concat(dfs)

In [85]:
df_Y4S = fileY4S.arrays(library="pd")

In [86]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])

9865292
211319


In [87]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [88]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])

9754944
211319


## take a sample if used in notebook for faster processing

In [95]:
#df_Y4Ssample = df_Y4S.sample(n=10000)
#df_FSPssample = df_FSPs[df_FSPs['__event__'].isin(df_Y4Ssample["__event__"])]

In [96]:
#df_Y4S=df_Y4Ssample
#df_FSPs=df_FSPssample

In [91]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("df_Y4S.shape[0]:",df_Y4S.shape[0])

462793
10000


### delete particles which occur more than ones based on uniqueParticleIdentifier

In [92]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsFSPs_uniqParID.sort_values("count"))

In [93]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("groupsFSPs_uniqParID.shape[0]:",groupsFSPs_uniqParID.shape[0])
print("event_Bs.shape[0]:",event_Bs.shape[0])

462793
343589
9983


In [94]:
# delete particles which occur more than ones (keep first) and if possible keep the one with basf2_used==1
print("df_FSPs["basf2_used"].value_counts():",df_FSPs["basf2_used"].value_counts())
df_FSPs = df_FSPs.sort_values("basf2_used",ascending=False).drop_duplicates(subset=("__event__","uniqueParticleIdentifier"), keep='first')
print("df_FSPs["basf2_used"].value_counts():",df_FSPs["basf2_used"].value_counts())

0.0    246478
1.0    216315
Name: basf2_used, dtype: int64
1.0    216315
0.0    127274
Name: basf2_used, dtype: int64


In [58]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])

343386


## check if category combinations make sense

In [59]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,127411
1,1.0,0.0,0.0,38506
2,1.0,0.0,1.0,108556
3,1.0,1.0,0.0,68913


### add two cols with extra info

In [60]:
# function to create col with the particles mother B's uniqueParticleIdentifier
def B_ID(s):
    label = 0
    for i in range(10): 
        mcMotheri_uniqParID = "mcMother{}_uniqParID".format(i)
        if ((s[mcMotheri_uniqParID]) == 83886082.0):
            label = 83886082   
        elif ((s[mcMotheri_uniqParID]) == 83886081.0):
            label = 83886081   
    return label
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)


In [61]:
# if particle was used by basf2 but neither for B-sig or X it is from the Hc
def Hc(s):
    label = 0
    if ((s["basf2_used"] == 1.0) & (s["basf2_Bsig"] == 0.0) & (s["basf2_X"] == 0.0)):
            label = 1   
    
    return label
df_FSPs['Hc'] = df_FSPs.apply(Hc, axis=1)


In [82]:
# this shows that Hc is sometimes combined from both B's which is of course wrong
#groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","B_ID","Hc","basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsAllFSPs.sort_values("Hc"))

## data preprocessing

### create df with per event info about which B is sig and which is tag

In [62]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby(["__event__","B_ID","Hc"]).size(),
                             'sum_p': df_FSPs.groupby(["__event__","B_ID","Hc"])["p"].sum()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsAllFSPs[(groupsAllFSPs["__event__"] == 3183239)].sort_values("__event__"))

In [63]:
#print(groupsAllFSPs[groupsAllFSPs["Hc"] == 1].sort_values("__event__"))

In [64]:
events=[]
B_tag_IDs=[]
B_sig_IDs=[]

In [65]:
unclearHc = 0
for evt in pd.unique(groupsAllFSPs[groupsAllFSPs["Hc"] == 1]["__event__"]):
    # sort by sum_p to take "max_count_idx" from the H_c particles with more momentum if two categories have the same particle count
    singleEvt_Hcs = groupsAllFSPs[(groupsAllFSPs["Hc"] == 1) & (groupsAllFSPs["__event__"] == evt)].sort_values("sum_p",ascending=False)
    
    # B_ID=0 => background, so take the other one if available
    singleEvt_Hcs = singleEvt_Hcs[(singleEvt_Hcs["B_ID"] != 0)]
    
    if singleEvt_Hcs.empty:
        unclearHc += 1
        continue
    
    max_count_idx = singleEvt_Hcs["count"].idxmax()
    
    #print((max_count_idx))
    max_count_row = singleEvt_Hcs.loc[[max_count_idx]]
       
    
    #print(max_count_row,'\n\n\n\n')

        
    events.append(max_count_row.iloc[0]['__event__'])
    B_tag_IDs.append(max_count_row.iloc[0]['B_ID']) # this is Btag because it is Hc's mother B
    Bsig_tmp = 0
    if B_tag_IDs[-1] == 83886082:
        Bsig_tmp = 83886081.0
    elif B_tag_IDs[-1] == 83886081:
        Bsig_tmp = 83886082.0
    #else:
    #    unclearHc +=1
    #    print(singleEvt_Hcs)
    #    print(events[-1],B_tag_IDs[-1],max_count_idx)
    #    raise ValueError('Btag/Bsig assignment unclear')
    B_sig_IDs.append(Bsig_tmp)
    
print("in",unclearHc,"cases, Btag (B mother of Hc) was unclear")
print("equals to",round(unclearHc/(len(B_sig_IDs)+unclearHc) , 4)*100,"%")
    

in 0 cases, Btag (B mother of Hc) was unclear
equals to 0.0 %


In [67]:
event_Bs = pd.DataFrame(
{"__event__" : events,
"B_tag_ID" : B_tag_IDs,
"B_sig_ID" : B_sig_IDs})

In [68]:
# throw away events with unclear Btag
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(event_Bs["__event__"])]
print("df_FSPs.shape[0]:",df_FSPs.shape[0])

343386
342986


In [69]:
event_Bs[:10]

,__event__,B_tag_ID,B_sig_ID
0,4094.0,83886081.0,83886082.0
1,4358.0,83886081.0,83886082.0
2,8765.0,83886082.0,83886081.0
3,9687.0,83886081.0,83886082.0
4,10725.0,83886081.0,83886082.0
5,10796.0,83886081.0,83886082.0
6,10842.0,83886082.0,83886081.0
7,13458.0,83886082.0,83886081.0
8,13915.0,83886081.0,83886082.0
9,13987.0,83886081.0,83886082.0


In [70]:
# check that B-tag and B-sig are not equal for any event -> only 2 rows shall appear here
pd.DataFrame({'count' : event_Bs.groupby(["B_tag_ID","B_sig_ID"]).size()}).reset_index()

,B_tag_ID,B_sig_ID,count
0,83886081.0,83886082.0,5034
1,83886082.0,83886081.0,4949


### save dataframes on NFS

In [71]:
df_FSPs.to_csv(root_path + "df_FSPs_preProcessed.csv")
event_Bs.to_csv(root_path + "event_Bs.csv")

### load dataframes

In [6]:
df_FSPs = pd.read_csv(root_path + "df_FSPs_preProcessed.csv")
event_Bs = pd.read_csv(root_path + "event_Bs.csv")

### delete Hc particles, after loading data, so saved df's cotain the Hc particles as well

In [72]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
df_FSPs_final = df_FSPs[df_FSPs["Hc"] == 0]
print("df_FSPs_final.shape[0]:",df_FSPs_final.shape[0])

342986
304480


In [73]:
event_Bs.keys()

Index(['__event__', 'B_tag_ID', 'B_sig_ID'], dtype='object')

In [74]:
df_FSPs_final.keys()

Index(['__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'basf2_X', 'basf2_used', 'basf2_Bsig',
       'isSignal', 'uniqueParticleIdentifier', 'mcErrors', 'mcPDG',
       'genMotherID', 'genMotherP', 'genMotherPDG', 'px', 'py', 'pz', 'pt',
       'p', 'E', 'kaonID', 'pionID', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'mcMother0_uniqParID', 'mcMother1_uniqParID', 'mcMother2_uniqParID',
       'mcMother3_uniqParID', 'mcMother4_uniqParID', 'mcMother5_uniqParID',
       'mcMother6_uniqParID', 'mcMother7_uniqParID', 'mcMother8_uniqParID',
       'mcMother9_uniqParID', 'PDG', 'B_ID', 'Hc'],
      dtype='object')

# start of NN data creation

In [75]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 11
maxFSPs: 58 



<ipython-input-75-887bf7860262>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')


In [76]:
data_dir = Path(nfs_path + "NNdata/" + data_subdir + root_subdir)    
data_dir.mkdir(parents=True, exist_ok=True)
print("Will save data to:", data_dir,'is', save_data ,'\n')

Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NNdata/Dstlnu_Hc_corr_BsigX_separation_dataRun1/axheim_data2_MC14_100kEvts is True 



In [77]:
for num_FSPs_toData in range(minFSPs, maxFSPs+1):
    df_num_subset = df_FSPs_final.copy()
    df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]
    
        
    numEvents = df_num_subset.__event__.nunique()
    print("numEvents:",numEvents)
    print("num_FSPs_toData:",num_FSPs_toData)  
    if numEvents == 0:
        print("skipped because empty \n")
        continue

    if numEvents < 10:
        print("skipped because <10 events \n")
        continue
    
    num_features = 4
    leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
    SA_target =  np.zeros((numEvents, num_FSPs_toData))
    global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)
    
    event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
    #print("len(event_list):",len(event_list))
    for i in range(numEvents):

        event_iter = event_list[i]

        global_tag_masterInfo = "evt" + str(event_iter)
        global_tag[i,-1] = global_tag_masterInfo
        #print("global_tag[i,-1]:",global_tag[i,-1])
        #print("i:",i,"event_iter:",event_iter)

        event_df = df_num_subset[df_num_subset.__event__ == event_iter]

        for j in range(num_FSPs_toData):
            #print("numParticle:",j)
            particle = event_df.iloc[j]

            #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
            leaves[i,j,0] = particle["px"]
            leaves[i,j,1] = particle["py"]
            leaves[i,j,2] = particle["pz"]
            leaves[i,j,3] = particle["E"]
            
            basf2_usage = "basf2_NONE"
            if particle["basf2_Bsig"] == 1.0:
                basf2_usage = "basf2_Bsig"
            elif particle["basf2_X"] == 1.0:
                basf2_usage = "basf2_X"
            elif particle["basf2_used"] == 0:
                basf2_usage = "basf2_bg"

            global_tag_Info = str((particle["mcPDG"])) 
            global_tag_Info += "_" + basf2_usage
            global_tag[i,j] = global_tag_Info

            label = -10 # error code if assignment fails
            B_tag_uniqID = event_Bs[event_Bs.__event__ == event_iter].iloc[0]['B_tag_ID']
            B_sig_uniqID = event_Bs[event_Bs.__event__ == event_iter].iloc[0]['B_sig_ID']
            if particle["B_ID"] == B_tag_uniqID:
                label = 1 # particle belongs to X (MC truth)
            elif particle["B_ID"] == B_sig_uniqID:
                label = 2 # particle belongs to Bsig (MC truth)
            elif particle["B_ID"] == 0:
                label = 0 # background
            
            
            SA_target[i,j] = label
            
        del event_df
        
        
    # shuffle the data    
    for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
        perms = np.random.permutation(leaves.shape[1])

        leaves[idx,:] = leaves[idx,perms]
        SA_target[idx,:] = SA_target[idx,perms]
        global_tag[idx,0:-1] = global_tag[idx,perms]
        
        
         


    #print(global_tag)
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    print("leaves.shape:",leaves.shape)
    print("SA_target.shape:",SA_target.shape)
    print("global_tag.shape:",global_tag.shape)


    print("leaves[0]:",leaves[0])
    print("SA_target[0]:",SA_target[0])
    print("global_tag[0]:",global_tag[0])

    x=leaves
    y=SA_target
    z=global_tag

    x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
    x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

    if save_data==True:
        np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
        np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
        np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

        np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
        np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
        np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

        np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
        np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
        np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)

    
    print("")
    #del df_num_subset


    del df_num_subset
                                          

numEvents: 1
num_FSPs_toData: 11
skipped because <10 events 

numEvents: 0
num_FSPs_toData: 12
skipped because empty 

numEvents: 3
num_FSPs_toData: 13
skipped because <10 events 

numEvents: 8
num_FSPs_toData: 14
skipped because <10 events 

numEvents: 11
num_FSPs_toData: 15
leaves.shape: (11, 15, 4)
SA_target.shape: (11, 15)
global_tag.shape: (11, 16)
leaves[0]: [[-2.75482088e-01 -5.19157946e-01  4.72412676e-01  9.01280234e-01]
 [ 3.29472512e-01  4.04790223e-01 -2.10221931e-01  5.79724429e-01]
 [-1.05169946e-02  5.96350692e-02  7.14062229e-02  1.68064540e-01]
 [-1.72787663e-02  2.16576234e-02 -4.80147963e-03  2.81187236e-02]
 [-6.85140863e-02 -2.33237483e-02 -1.15756376e-03  7.23845095e-02]
 [ 6.04003528e-03  9.98164713e-03 -2.25332882e-02  2.53744829e-02]
 [ 8.65890145e-01 -4.58193749e-01 -5.24339557e-01  1.11987460e+00]
 [ 1.91684633e-01 -5.27542196e-02 -1.03206322e-01  2.24003463e-01]
 [-1.03996241e+00  1.34095913e-02  1.53371537e+00  1.85611103e+00]
 [-6.61818776e-03  1.96104217e


numEvents: 126
num_FSPs_toData: 20
leaves.shape: (126, 20, 4)
SA_target.shape: (126, 20)
global_tag.shape: (126, 21)
leaves[0]: [[-0.96612293 -0.69614768 -0.16445683  1.29952996]
 [ 0.01267718  0.01218023 -0.01116726  0.0208273 ]
 [ 0.01362544  0.01584664 -0.03995909  0.04509432]
 [ 0.16520719 -0.03818807 -0.00603573  0.16967078]
 [ 0.21983039  0.01895103  0.1739092   0.5680193 ]
 [-0.17497464 -0.17515962 -0.02761157  0.24911727]
 [-0.03179014  0.02769185  0.01196861  0.04382578]
 [ 0.4232778  -0.82459855 -0.32560709  0.982419  ]
 [ 0.04178032  0.16452479  0.08267161  0.18880836]
 [-0.00753863 -0.01473701 -0.02140075  0.02705555]
 [-0.11683015 -0.00678951  0.04869025  0.12675221]
 [ 0.54257864 -0.47477835  0.05949088  0.73676667]
 [-0.0307834  -0.30103326  0.62320679  0.70670732]
 [-0.0945418  -0.0207451   0.06527913  0.18196089]
 [-0.47885162  0.17540298 -0.93377984  1.0730749 ]
 [ 0.29388571  0.32861364  0.86159456  1.08647038]
 [ 0.00964431  0.02382471  0.01688296  0.03075165]
 [-0

KeyboardInterrupt: 

In [100]:
print("saving is done")
now = datetime.now()
print("time at end =", now)

saving is done
time at end = 2021-09-20 15:13:30.136619
